In [1]:
import pandas as pd
import numpy as np
import re
import polars as pl
import gcsfs

In [2]:
rkgnodes = pl.read_parquet("gs://mtrx-us-central1-hub-dev-storage/kedro/data/tests/gt_retained_robo30fd_run/datasets/ingestion/int/robokop/30fd1bfc18cd5ccb/nodes/")
print(rkgnodes.head)

<bound method DataFrame.head of shape: (4_663_956, 1_963)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ id        ┆ name:stri ┆ category  ┆ equivalen ┆ … ┆ MONDO_SUP ┆ MONDO_SUP ┆ taxon:str ┆ NCBITaxo │
│ ---       ┆ ng        ┆ ---       ┆ t_identif ┆   ┆ ERCLASS_r ┆ ERCLASS_p ┆ ing       ┆ n:string │
│ str       ┆ ---       ┆ str       ┆ iers      ┆   ┆ enal_dise ┆ erceptual ┆ ---       ┆ ---      │
│           ┆ str       ┆           ┆ ---       ┆   ┆ ase…      ┆ _di…      ┆ str       ┆ str      │
│           ┆           ┆           ┆ str       ┆   ┆ ---       ┆ ---       ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ str       ┆ str       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ CHEBI:10  ┆ (+)-Ather ┆ biolink:S ┆ CHEBI:10| ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆ ospermoli ┆ mallMol

In [3]:
cols = list(rkgnodes.columns)
rcols = [item for item in cols if "CHEBI" not in item and "MONDO" not in item and "int" not in item]
print(len(rcols))
rkgnodes = rkgnodes.select(rcols)


21


In [4]:
rkgnodes.shape

(4663956, 21)

In [5]:
rkgnodes.head()

id,name:string,category,equivalent_identifiers,information_content:float,description:string,cd_formula:string,cd_molweight:float,clogp:float,alogs:float,tpsa:float,mrdef:string,smiles:string,status:string,definition:string,url:string,locus_group:string,symbol:string,location:string,taxon:string,NCBITaxon:string
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""CHEBI:10""","""(+)-Atherospermoline""","""biolink:SmallMolecule|biolink:…","""CHEBI:10|PUBCHEM.COMPOUND:1771…","""100.0""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:100""","""(-)-medicarpin""","""biolink:NamedThing|biolink:Phy…","""CHEMBL.COMPOUND:CHEMBL238845|C…","""95.4""","""The (-)-enantiomer of medicarp…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:10000""","""Vismione D""","""biolink:SmallMolecule|biolink:…","""CHEBI:10000|PUBCHEM.COMPOUND:5…","""100.0""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:100000""","""(2S,3S,4R)-3-[4-(3-cyclopentyl…","""biolink:SmallMolecule|biolink:…","""CHEBI:100000|PUBCHEM.COMPOUND:…","""100.0""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:100001""","""N-[(2R,3S,6R)-2-(hydroxymethyl…","""biolink:SmallMolecule|biolink:…","""CHEBI:100001|PUBCHEM.COMPOUND:…","""100.0""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [6]:
rkgnodes.filter(pl.col("id") == "CHEBI:50745")

id,name:string,category,equivalent_identifiers,information_content:float,description:string,cd_formula:string,cd_molweight:float,clogp:float,alogs:float,tpsa:float,mrdef:string,smiles:string,status:string,definition:string,url:string,locus_group:string,symbol:string,location:string,taxon:string,NCBITaxon:string
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""CHEBI:50745""","""progestogen""","""biolink:NamedThing|biolink:Phy…","""CHEBI:59826|CHEBI:50745|MESH:D…","""78.5""","""A compound that interacts with…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [7]:
gt_retained_ROBOKOP = pl.read_parquet("gs://mtrx-us-central1-hub-dev-storage/kedro/data/tests/base_robo_num_walks_50_walk_length_30/runs/main-0b4f8ab2/datasets/modelling/ground_truth/translator/v2.10.0_validated/ground_truth_pairs/")
gt_retained_ROBOKOP

original_object,original_subject,indication,contraindication,y,drug|disease,disease,drug,predicate,source,subject_normalization_success,target,object_normalization_success
str,str,str,str,i32,str,str,str,str,str,bool,str,bool
"""MONDO:0002406""","""CHEBI:31199""","""True""","""False""",1,"""CHEBI:31199|MONDO:0002406""","""MONDO:0002406""","""CHEBI:31199""","""clinical_trails""","""CHEBI:31199""",true,"""MONDO:0002406""",true
"""MONDO:0002406""","""CHEBI:5108""","""True""","""False""",1,"""CHEBI:5108|MONDO:0002406""","""MONDO:0002406""","""CHEBI:5108""","""clinical_trails""","""CHEBI:5108""",true,"""MONDO:0002406""",true
"""MONDO:0002406""","""CHEBI:2948""","""True""","""False""",1,"""CHEBI:2948|MONDO:0002406""","""MONDO:0002406""","""CHEBI:2948""","""clinical_trails""","""CHEBI:2948""",true,"""MONDO:0002406""",true
"""MONDO:0002406""","""CHEBI:422""","""True""","""False""",1,"""CHEBI:422|MONDO:0002406""","""MONDO:0002406""","""CHEBI:422""","""clinical_trails""","""CHEBI:422""",true,"""MONDO:0002406""",true
"""MONDO:0002406""","""CHEBI:31414""","""True""","""False""",1,"""CHEBI:31414|MONDO:0002406""","""MONDO:0002406""","""CHEBI:31414""","""clinical_trails""","""CHEBI:31414""",true,"""MONDO:0002406""",true
…,…,…,…,…,…,…,…,…,…,…,…,…
"""MONDO:0033312""","""CHEBI:84500""","""False""","""True""",0,"""CHEBI:84500|MONDO:0033312""","""MONDO:0033312""","""CHEBI:84500""","""clinical_trails""","""CHEBI:84500""",true,"""MONDO:0033312""",true
"""MONDO:0033312""","""CHEBI:47499""","""False""","""True""",0,"""CHEBI:47499|MONDO:0033312""","""MONDO:0033312""","""CHEBI:47499""","""clinical_trails""","""CHEBI:47499""",true,"""MONDO:0033312""",true
"""MONDO:0033312""","""CHEBI:47781""","""False""","""True""",0,"""CHEBI:47781|MONDO:0033312""","""MONDO:0033312""","""CHEBI:47781""","""clinical_trails""","""CHEBI:47781""",true,"""MONDO:0033312""",true


In [8]:
gt_retained_ROBOKOP = gt_retained_ROBOKOP.with_columns((pl.col("original_subject") == pl.col("drug")).alias("ori_vs_drugid"))

In [9]:
gt_retained_ROBOKOP

original_object,original_subject,indication,contraindication,y,drug|disease,disease,drug,predicate,source,subject_normalization_success,target,object_normalization_success,ori_vs_drugid
str,str,str,str,i32,str,str,str,str,str,bool,str,bool,bool
"""MONDO:0002406""","""CHEBI:31199""","""True""","""False""",1,"""CHEBI:31199|MONDO:0002406""","""MONDO:0002406""","""CHEBI:31199""","""clinical_trails""","""CHEBI:31199""",true,"""MONDO:0002406""",true,true
"""MONDO:0002406""","""CHEBI:5108""","""True""","""False""",1,"""CHEBI:5108|MONDO:0002406""","""MONDO:0002406""","""CHEBI:5108""","""clinical_trails""","""CHEBI:5108""",true,"""MONDO:0002406""",true,true
"""MONDO:0002406""","""CHEBI:2948""","""True""","""False""",1,"""CHEBI:2948|MONDO:0002406""","""MONDO:0002406""","""CHEBI:2948""","""clinical_trails""","""CHEBI:2948""",true,"""MONDO:0002406""",true,true
"""MONDO:0002406""","""CHEBI:422""","""True""","""False""",1,"""CHEBI:422|MONDO:0002406""","""MONDO:0002406""","""CHEBI:422""","""clinical_trails""","""CHEBI:422""",true,"""MONDO:0002406""",true,true
"""MONDO:0002406""","""CHEBI:31414""","""True""","""False""",1,"""CHEBI:31414|MONDO:0002406""","""MONDO:0002406""","""CHEBI:31414""","""clinical_trails""","""CHEBI:31414""",true,"""MONDO:0002406""",true,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MONDO:0033312""","""CHEBI:84500""","""False""","""True""",0,"""CHEBI:84500|MONDO:0033312""","""MONDO:0033312""","""CHEBI:84500""","""clinical_trails""","""CHEBI:84500""",true,"""MONDO:0033312""",true,true
"""MONDO:0033312""","""CHEBI:47499""","""False""","""True""",0,"""CHEBI:47499|MONDO:0033312""","""MONDO:0033312""","""CHEBI:47499""","""clinical_trails""","""CHEBI:47499""",true,"""MONDO:0033312""",true,true
"""MONDO:0033312""","""CHEBI:47781""","""False""","""True""",0,"""CHEBI:47781|MONDO:0033312""","""MONDO:0033312""","""CHEBI:47781""","""clinical_trails""","""CHEBI:47781""",true,"""MONDO:0033312""",true,true


In [10]:
gt_retained_ROBOKOP.select(pl.col("ori_vs_drugid").value_counts())

ori_vs_drugid
struct[2]
"{true,46065}"


In [11]:
gt_retained_ROBOKOP.shape

(46065, 14)

In [12]:
fs = gcsfs.GCSFileSystem()
files = fs.ls('mtrx-us-central1-hub-dev-storage/kedro/data/tests/base_robo_num_walks_50_walk_length_30/datasets/ingestion/ground_truth/translator/v2.10.0_validated/gt_pairs.tsv')

# Filter to only .tsv files
tsv_files = ['gs://' + f for f in files if f.endswith('.csv')]

# Read and concatenate all TSVs
gt_ori = pd.concat([pd.read_csv(f, sep='\t') for f in tsv_files], ignore_index=True)

gt_ori = pl.from_pandas(gt_ori)

In [13]:
gt_ori

source,target,indication,contraindication,y,drug|disease
str,str,bool,bool,i64,str
"""UMLS:C0450442""","""MONDO:0005429""",true,false,1,"""UMLS:C0450442|MONDO:0005429"""
"""CHEBI:8711""","""MONDO:0005429""",true,false,1,"""CHEBI:8711|MONDO:0005429"""
"""CHEBI:59826""","""HP:0100608""",true,false,1,"""CHEBI:59826|HP:0100608"""
"""UMLS:C0243192""","""MONDO:0000088""",true,false,1,"""UMLS:C0243192|MONDO:0000088"""
"""PUBCHEM.COMPOUND:36523""","""MONDO:0000088""",true,false,1,"""PUBCHEM.COMPOUND:36523|MONDO:0…"
…,…,…,…,…,…
"""CHEBI:45863""","""MONDO:0019210""",false,true,0,"""CHEBI:45863|MONDO:0019210"""
"""CHEBI:3699""","""MONDO:0001942""",false,true,0,"""CHEBI:3699|MONDO:0001942"""
"""CHEBI:6801""","""MONDO:0001942""",false,true,0,"""CHEBI:6801|MONDO:0001942"""


In [14]:
gt_ori["y"].value_counts()

y,count
i64,u32
0,40439
1,28562


In [15]:
gt = gt_ori.with_columns(
    pl.col("drug|disease").is_in(gt_retained_ROBOKOP["drug|disease"]).alias("ddpair_in_ROBOKOP")
)

/var/folders/1r/m2w57_0j69n3bgl22609_d580000gq/T/ipykernel_94107/2567263648.py:1: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  gt = gt_ori.with_columns(


In [16]:
gt["ddpair_in_ROBOKOP"].value_counts()

ddpair_in_ROBOKOP,count
bool,u32
true,46065
false,22936


In [17]:
gt = gt.with_columns(pl.col("source").is_in(rkgnodes["id"]).alias("source_id_in_ROBOKOP_id"))
gt = gt.with_columns(pl.col("target").is_in(rkgnodes["id"]).alias("target_id_in_ROBOKOP_id"))

/var/folders/1r/m2w57_0j69n3bgl22609_d580000gq/T/ipykernel_94107/2490421795.py:1: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  gt = gt.with_columns(pl.col("source").is_in(rkgnodes["id"]).alias("source_id_in_ROBOKOP_id"))
/var/folders/1r/m2w57_0j69n3bgl22609_d580000gq/T/ipykernel_94107/2490421795.py:2: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  gt = gt.with_columns(pl.col("target").is_in(rkgnodes["id"]).alias("target_id_in_ROBOKOP_id"))


In [18]:
rkgnodes_exploded = rkgnodes.with_columns(
    pl.col("equivalent_identifiers").str.split("|")
).explode("equivalent_identifiers")

In [19]:
gt = gt.join(
    rkgnodes_exploded.select(["id", "equivalent_identifiers"]),
    left_on="source",
    right_on="equivalent_identifiers",
    how="left"
).rename({"id": "source_id_in_ROBOKOP_EquiIDS"})

In [20]:
gt = gt.join(
    rkgnodes_exploded.select(["id", "equivalent_identifiers"]),
    left_on="target",
    right_on="equivalent_identifiers",
    how="left"
).rename({"id": "target_id_in_ROBOKOP_EquiIDS"})

In [21]:
# source id not preferred id but FOUND in equivalent identifiers
gt.filter(
    (pl.col("source_id_in_ROBOKOP_id") == False) &
    (pl.col("source_id_in_ROBOKOP_EquiIDS").is_not_null())
)#.select(pl.col("source").n_unique())

source,target,indication,contraindication,y,drug|disease,ddpair_in_ROBOKOP,source_id_in_ROBOKOP_id,target_id_in_ROBOKOP_id,source_id_in_ROBOKOP_EquiIDS,target_id_in_ROBOKOP_EquiIDS
str,str,bool,bool,i64,str,bool,bool,bool,str,str
"""CHEBI:59826""","""HP:0100608""",true,false,1,"""CHEBI:59826|HP:0100608""",false,false,true,"""CHEBI:50745""","""HP:0100608"""
"""PUBCHEM.COMPOUND:23686432""","""UMLS:C0038663""",true,false,1,"""PUBCHEM.COMPOUND:23686432|UMLS…",false,false,true,"""CHEBI:30146""","""UMLS:C0038663"""
"""CHEBI:4463""","""MONDO:0017364""",true,false,1,"""CHEBI:4463|MONDO:0017364""",false,false,true,"""CHEBI:4462""","""MONDO:0017364"""
"""CHEBI:100241""","""MONDO:0019345""",true,false,1,"""CHEBI:100241|MONDO:0019345""",false,false,true,"""CHEBI:59936""","""MONDO:0019345"""
"""CHEBI:29007""","""MONDO:0019345""",true,false,1,"""CHEBI:29007|MONDO:0019345""",false,false,true,"""CHEBI:3514""","""MONDO:0019345"""
…,…,…,…,…,…,…,…,…,…,…
"""CHEBI:4027""","""MONDO:0008383""",false,true,0,"""CHEBI:4027|MONDO:0008383""",false,false,true,"""CHEBI:4026""","""MONDO:0008383"""
"""CHEBI:47381""","""MONDO:0004795""",false,true,0,"""CHEBI:47381|MONDO:0004795""",false,false,true,"""CHEBI:4508""","""MONDO:0004795"""
"""PUBCHEM.COMPOUND:60852""","""HP:0012531""",false,true,0,"""PUBCHEM.COMPOUND:60852|HP:0012…",false,false,true,"""UNII:UMD7G2653W""","""HP:0012531"""


In [22]:
# target id not preferred id but FOUND in equivalent identifiers
gt.filter(
    (pl.col("target_id_in_ROBOKOP_id") == False) &
    (pl.col("target_id_in_ROBOKOP_EquiIDS").is_not_null())
)#.select(pl.col("target").n_unique())

source,target,indication,contraindication,y,drug|disease,ddpair_in_ROBOKOP,source_id_in_ROBOKOP_id,target_id_in_ROBOKOP_id,source_id_in_ROBOKOP_EquiIDS,target_id_in_ROBOKOP_EquiIDS
str,str,bool,bool,i64,str,bool,bool,bool,str,str
"""CHEBI:6909""","""UMLS:C0031094""",true,false,1,"""CHEBI:6909|UMLS:C0031094""",false,true,false,"""CHEBI:6909""","""EFO:1001393"""
"""UMLS:C1136254""","""UMLS:C0031094""",true,false,1,"""UMLS:C1136254|UMLS:C0031094""",false,true,false,"""UMLS:C1136254""","""EFO:1001393"""
"""CHEBI:16664""","""UMLS:C0019911""",true,false,1,"""CHEBI:16664|UMLS:C0019911""",false,true,false,"""CHEBI:16664""","""MONDO:0005799"""
"""CHEBI:6704""","""UMLS:C0019911""",true,false,1,"""CHEBI:6704|UMLS:C0019911""",false,true,false,"""CHEBI:6704""","""MONDO:0005799"""
"""CHEBI:136007""","""NCIT:C148034""",true,false,1,"""CHEBI:136007|NCIT:C148034""",false,true,false,"""CHEBI:136007""","""MONDO:0100187"""
…,…,…,…,…,…,…,…,…,…,…
"""CHEBI:3611""","""EFO:0009454""",false,true,0,"""CHEBI:3611|EFO:0009454""",false,true,false,"""CHEBI:3611""","""MONDO:0001126"""
"""CHEBI:6129""","""EFO:0009454""",false,true,0,"""CHEBI:6129|EFO:0009454""",false,true,false,"""CHEBI:6129""","""MONDO:0001126"""
"""CHEBI:3219""","""HP:0002039""",false,true,0,"""CHEBI:3219|HP:0002039""",false,true,false,"""CHEBI:3219""","""MONDO:0005351"""


In [23]:
# source id neither in preferred id nor in equivalent identifiers
gt.filter(
    (pl.col("source_id_in_ROBOKOP_id") == False) &
    (pl.col("source_id_in_ROBOKOP_EquiIDS").is_null())
)#.select(pl.col("source").n_unique())

source,target,indication,contraindication,y,drug|disease,ddpair_in_ROBOKOP,source_id_in_ROBOKOP_id,target_id_in_ROBOKOP_id,source_id_in_ROBOKOP_EquiIDS,target_id_in_ROBOKOP_EquiIDS
str,str,bool,bool,i64,str,bool,bool,bool,str,str
"""UMLS:C0450442""","""MONDO:0005429""",true,false,1,"""UMLS:C0450442|MONDO:0005429""",false,false,true,null,"""MONDO:0005429"""
"""UMLS:C0243192""","""MONDO:0000088""",true,false,1,"""UMLS:C0243192|MONDO:0000088""",false,false,true,null,"""MONDO:0000088"""
"""UMLS:C0003289""","""UMLS:C0038663""",true,false,1,"""UMLS:C0003289|UMLS:C0038663""",false,false,true,null,"""UMLS:C0038663"""
"""UMLS:C0003289""","""UMLS:C0563664""",true,false,1,"""UMLS:C0003289|UMLS:C0563664""",false,false,false,null,null
"""UMLS:C1611640""","""MONDO:0007739""",true,false,1,"""UMLS:C1611640|MONDO:0007739""",false,false,true,null,"""MONDO:0007739"""
…,…,…,…,…,…,…,…,…,…,…
"""PUBCHEM.COMPOUND:49800036""","""DOID:0060504""",false,true,0,"""PUBCHEM.COMPOUND:49800036|DOID…",false,false,false,null,null
"""PUBCHEM.COMPOUND:52945526""","""MONDO:0001475""",false,true,0,"""PUBCHEM.COMPOUND:52945526|MOND…",false,false,true,null,"""MONDO:0001475"""
"""PUBCHEM.COMPOUND:14888""","""MONDO:0002447""",false,true,0,"""PUBCHEM.COMPOUND:14888|MONDO:0…",false,false,true,null,"""MONDO:0002447"""


In [24]:
# target id neither in preferred id nor in equivalent identifiers
gt.filter(
    (pl.col("target_id_in_ROBOKOP_id") == False) &
    (pl.col("target_id_in_ROBOKOP_EquiIDS").is_null())
)#.select(pl.col("target").n_unique())

source,target,indication,contraindication,y,drug|disease,ddpair_in_ROBOKOP,source_id_in_ROBOKOP_id,target_id_in_ROBOKOP_id,source_id_in_ROBOKOP_EquiIDS,target_id_in_ROBOKOP_EquiIDS
str,str,bool,bool,i64,str,bool,bool,bool,str,str
"""CHEBI:46345""","""UMLS:C1271404""",true,false,1,"""CHEBI:46345|UMLS:C1271404""",false,true,false,"""CHEBI:46345""",null
"""UMLS:C0003289""","""UMLS:C0563664""",true,false,1,"""UMLS:C0003289|UMLS:C0563664""",false,false,false,null,null
"""UMLS:C0003232""","""UMLS:C0264222""",true,false,1,"""UMLS:C0003232|UMLS:C0264222""",false,false,false,null,null
"""CHEBI:6432""","""UMLS:C0554021""",true,false,1,"""CHEBI:6432|UMLS:C0554021""",false,true,false,"""CHEBI:6432""",null
"""CHEBI:74947""","""UMLS:C0554021""",true,false,1,"""CHEBI:74947|UMLS:C0554021""",false,true,false,"""CHEBI:74947""",null
…,…,…,…,…,…,…,…,…,…,…
"""CHEBI:31717""","""DOID:0060504""",false,true,0,"""CHEBI:31717|DOID:0060504""",false,true,false,"""CHEBI:31717""",null
"""CHEBI:2686""","""DOID:0060504""",false,true,0,"""CHEBI:2686|DOID:0060504""",false,true,false,"""CHEBI:2686""",null
"""CHEBI:50305""","""DOID:0060504""",false,true,0,"""CHEBI:50305|DOID:0060504""",false,true,false,"""CHEBI:50305""",null


In [ ]:
gt.filter(pl.col("source")=="UMLS:C0450442")

source,target,indication,contraindication,y,drug|disease,ddpair_in_ROBOKOP,source_id_in_ROBOKOP_id,target_id_in_ROBOKOP_id,source_id_in_ROBOKOP_EquiIDS,target_id_in_ROBOKOP_EquiIDS
str,str,bool,bool,i64,str,bool,bool,bool,str,str
"""UMLS:C0450442""","""MONDO:0005429""",true,false,1,"""UMLS:C0450442|MONDO:0005429""",false,false,true,null,"""MONDO:0005429"""
"""UMLS:C0450442""","""HP:0000855""",true,false,1,"""UMLS:C0450442|HP:0000855""",false,false,true,null,"""HP:0000855"""
"""UMLS:C0450442""","""MONDO:0044875""",true,false,1,"""UMLS:C0450442|MONDO:0044875""",false,false,true,null,"""MONDO:0044875"""
"""UMLS:C0450442""","""MONDO:0005485""",true,false,1,"""UMLS:C0450442|MONDO:0005485""",false,false,true,null,"""MONDO:0005485"""
"""UMLS:C0450442""","""MONDO:0020311""",true,false,1,"""UMLS:C0450442|MONDO:0020311""",false,false,true,null,"""MONDO:0020311"""
…,…,…,…,…,…,…,…,…,…,…
"""UMLS:C0450442""","""UMLS:C1519670""",true,false,1,"""UMLS:C0450442|UMLS:C1519670""",false,false,false,null,null
"""UMLS:C0450442""","""MONDO:0044067""",true,false,1,"""UMLS:C0450442|MONDO:0044067""",false,false,true,null,"""MONDO:0044067"""
"""UMLS:C0450442""","""MONDO:0044070""",true,false,1,"""UMLS:C0450442|MONDO:0044070""",false,false,true,null,"""MONDO:0044070"""


: 

In [ ]:
#gt.write_csv("ground_truth_list_from_rtxkg2_v2_10_0_breakdown_missing_condition_ROBOKOP_30fd.csv")

In [ ]:
gt.filter(~pl.col("ddpair_in_ROBOKOP") & pl.col("source_id_in_ROBOKOP_id") & pl.col("target_id_in_ROBOKOP_id"))

source,target,indication,contraindication,y,drug|disease,ddpair_in_ROBOKOP,source_id_in_ROBOKOP_id,target_id_in_ROBOKOP_id,source_id_in_ROBOKOP_EquiIDS,target_id_in_ROBOKOP_EquiIDS
str,str,bool,bool,i64,str,bool,bool,bool,str,str
"""MESH:D006062""","""MONDO:0000088""",true,false,1,"""MESH:D006062|MONDO:0000088""",false,true,true,"""MESH:D006062""","""MONDO:0000088"""
"""CHEBI:61115""","""MONDO:0007739""",true,false,1,"""CHEBI:61115|MONDO:0007739""",false,true,true,"""CHEBI:61115""","""MONDO:0007739"""
"""MESH:D000305""","""MONDO:0004769""",true,false,1,"""MESH:D000305|MONDO:0004769""",false,true,true,"""MESH:D000305""","""MONDO:0004769"""
"""CHEBI:35480""","""HP:0002027""",true,false,1,"""CHEBI:35480|HP:0002027""",false,true,true,"""CHEBI:35480""","""HP:0002027"""
"""CHEBI:24261""","""HP:0000855""",true,false,1,"""CHEBI:24261|HP:0000855""",false,true,true,"""CHEBI:24261""","""HP:0000855"""
…,…,…,…,…,…,…,…,…,…,…
"""UNII:J96U0ZD4Y6""","""MONDO:0009693""",false,true,0,"""UNII:J96U0ZD4Y6|MONDO:0009693""",false,true,true,"""UNII:J96U0ZD4Y6""","""MONDO:0009693"""
"""CHEBI:59349""","""MONDO:0005246""",false,true,0,"""CHEBI:59349|MONDO:0005246""",false,true,true,"""CHEBI:59349""","""MONDO:0005246"""
"""UNII:J96U0ZD4Y6""","""MONDO:0009891""",false,true,0,"""UNII:J96U0ZD4Y6|MONDO:0009891""",false,true,true,"""UNII:J96U0ZD4Y6""","""MONDO:0009891"""


In [23]:
rkgnodes.filter(pl.col("id") == "CHEBI:61115")["category"].to_list()

['biolink:ChemicalEntity|biolink:PhysicalEssence|biolink:ChemicalOrDrugOrTreatment|biolink:ChemicalEntityOrGeneOrGeneProduct|biolink:ChemicalEntityOrProteinOrPolypeptide|biolink:NamedThing|biolink:PhysicalEssenceOrOccurrent']

In [72]:
gt.select(pl.col("ddpair_in_ROBOKOP").value_counts())

ddpair_in_ROBOKOP
struct[2]
"{false,22936}"
"{true,46065}"


In [47]:
rkgnodes_exploded.filter(pl.col("id") == "CHEBI:50745")

id,name:string,category,equivalent_identifiers,information_content:float,description:string,cd_formula:string,cd_molweight:float,clogp:float,alogs:float,tpsa:float,mrdef:string,smiles:string,status:string,definition:string,url:string,locus_group:string,symbol:string,location:string,taxon:string,NCBITaxon:string
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""CHEBI:50745""","""progestogen""","""biolink:NamedThing|biolink:Phy…","""CHEBI:59826""","""78.5""","""A compound that interacts with…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:50745""","""progestogen""","""biolink:NamedThing|biolink:Phy…","""CHEBI:50745""","""78.5""","""A compound that interacts with…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:50745""","""progestogen""","""biolink:NamedThing|biolink:Phy…","""MESH:D011372""","""78.5""","""A compound that interacts with…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""CHEBI:50745""","""progestogen""","""biolink:NamedThing|biolink:Phy…","""UMLS:C0033306""","""78.5""","""A compound that interacts with…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
rkgnodes = rkgnodes[[]]

0          biolink:SmallMolecule|biolink:MolecularEntity|...
1          biolink:NamedThing|biolink:PhysicalEssence|bio...
2          biolink:SmallMolecule|biolink:MolecularEntity|...
3          biolink:SmallMolecule|biolink:MolecularEntity|...
4          biolink:SmallMolecule|biolink:MolecularEntity|...
                                 ...                        
4663951    biolink:Disease|biolink:DiseaseOrPhenotypicFea...
4663952    biolink:Disease|biolink:DiseaseOrPhenotypicFea...
4663953    biolink:Disease|biolink:DiseaseOrPhenotypicFea...
4663954    biolink:Disease|biolink:DiseaseOrPhenotypicFea...
4663955    biolink:Disease|biolink:DiseaseOrPhenotypicFea...
Name: category, Length: 4663956, dtype: object